# Follower network
We start from the Twitter follower network constructed for the paper [Right and left, partisanship predicts (asymmetric) vulnerability to misinformation](http://doi.org/10.37016/mr-2020-55). The following data files are available at https://doi.org/10.7910/DVN/6CZHH5:
* `anonymized-friends.json` 
* `measures.tab`

Briefly, this network was constructed as follows:
* We collected all tweets containing links (URLs) from a 10% random sample of public posts between June 1 and June 30, 2017, through the Twitter Decahose. 
* We selected all accounts that shared at least ten links from a set of news sources with known political valence (Bakshy et al., 2015). 
* We further selected those who shared at least one link from a source labeled as low-quality (https://github.com/BigMcLargeHuge/opensources). 
* We excluded likely bot accounts according to the BotometerLite classifier (Yang et al., 2020).

We keep the nodes with both partisanship and misinformation attributes, then we take the core of the network with approximately 10k nodes, and finally remove a random sample of edges to preserve the original average in/out-degree (number of friends/followers).

In [1]:
import networkx as nx
import csv
import json
import random
import importlib
import bot_model

In [2]:
path = "../EmpiricalNet_followers/"

In [13]:
# File has 3 columns: ID \t partisanship \t misinformation \n
partisanship = {}
misinformation = {}
with open(path + "measures.tab") as fd:
    rd = csv.reader(fd, delimiter="\t")
    next(rd) # skip header row
    for row in rd:
        partisanship[int(row[0])] = row[1]
        misinformation[int(row[0])] = row[2]

In [14]:
with open(path + 'anonymized-friends.json') as fp:
    adjlist = json.load(fp)

In [15]:
G = nx.DiGraph() 

In [16]:
# Directed network follower -> friend
for s in adjlist:
    n = int(s)
    if n in partisanship and n in misinformation:
        G.add_node(n, party=partisanship[n], misinfo=misinformation[n]) 
        for f in adjlist[s]:
            G.add_edge(n,f)

In [17]:
average_friends = G.number_of_edges() / G.number_of_nodes()
print("{} nodes and {} edges initially, with average number of friends {}".format(G.number_of_nodes(), G.number_of_edges(), average_friends))
friends = nx.subgraph(G, partisanship.keys())
print("{} nodes and {} edges after filtering".format(friends.number_of_nodes(), friends.number_of_edges()))

58048 nodes and 10499218 edges initially, with average number of friends 180.87131339581038
15056 nodes and 4327448 edges after filtering


In [19]:
# k-core decomposition until ~ 10k nodes in core
core_number = nx.core_number(friends)
nodes = friends.number_of_nodes()
k = 0
while nodes > 10000:
    k_core = nx.k_core(friends, k, core_number)
    nodes = k_core.number_of_nodes()
    k += 10
while nodes < 10000:
    k_core = nx.k_core(friends, k, core_number)
    nodes = k_core.number_of_nodes()
    k -= 1
print("{}-core has {} nodes, {} edges".format(k, k_core.number_of_nodes(), k_core.number_of_edges()))

94-core has 10006 nodes, 4144687 edges


In [20]:
# the network is super dense, so let us delete a random sample of edges
# we can set the initial average in/out-degree (average_friends) as a target 
friends_core = k_core.copy()
edges_to_keep = int(friends_core.number_of_nodes() * average_friends)
edges_to_delete = friends_core.number_of_edges() - edges_to_keep
deleted_edges = random.sample(friends_core.edges(), edges_to_delete)
friends_core.remove_edges_from(deleted_edges)
print("{}-core after edge-sampling has {} nodes, {} edges, and average number of friends {}".format(k, friends_core.number_of_nodes(), friends_core.number_of_edges(), friends_core.number_of_edges() / friends_core.number_of_nodes()))

94-core after edge-sampling has 10006 nodes, 1809798 edges, and average number of friends 180.8712772336598


In [22]:
nx.write_gml(friends_core, path + 'follower_network.gml')

## Test

In [52]:
importlib.reload(bot_model)
follower_net = bot_model.init_net(False, verbose=True, human_network = path + 'follower_network.gml', beta=0.01, gamma=0.001)
avg_quality = bot_model.simulation(False, network=follower_net, verbose=True, mu=0.5, phi=1, alpha=15)
print('average quality for follower network:', avg_quality)

Reading human network...
Generating bot network...
Merging human and bot networks...
Humans following bots...
time_steps =  0 , q =  200
time_steps =  1 , q =  0.4801671070272221
time_steps =  2 , q =  0.4931137924415558
average quality for follower network: 0.49038907607976123


## Experiments

In [29]:
# baseline:  alpha=15, beta=0, gamma=0, phi=1
# params:    alpha (15), beta (0.01), gamma (0.001), phi (1), flooding, network density
# targeting: hubs, partisans, conservatives, misinfo spreaders, core

In [11]:
importlib.reload(bot_model)

<module 'bot_model' from '/geode2/home/u030/fil/Carbonate/Xiaodan/Manipulating-Online-Marketplace-of-Ideas/bot_model.py'>

### Baseline

In [12]:
follower_net = bot_model.init_net(False, verbose=False, human_network = path + 'follower_network.gml', beta=0.0, gamma=0.0)
avg_quality_baseline = bot_model.simulation(False, network=follower_net, verbose=False, mu=0.5, phi=1, alpha=15)
print('average quality baseline:', avg_quality_baseline)

average quality baseline: 0.4803009094579953


### Experiment: alpha

In [ ]:
# alpha=15 from test above
bot_model.save_csv([15, 0.49038907607976123], csvfile = path + 'alpha.csv')
follower_net = bot_model.init_net(False, human_network = path + 'follower_network.gml', beta=0.01, gamma=0.001)

In [6]:
# note: using epsilon 0.01
# for alpha in [5, 45, 135]:
#  alpha = 5, average quality = 0.46635445323768177
#  alpha = 45, average quality = 0.5005471496662851
#  alpha = 135, average quality = 0.48341830865846264

# more values with epsilon successively set to 0.001
for alpha in [2, 4, 8, 16, 32, 64, 128]:
    avg_quality = bot_model.simulation(False, network=follower_net, verbose=False, mu=0.5, phi=1, alpha=alpha)
    print('alpha = {}, average quality = {}'.format(alpha, avg_quality))
    bot_model.save_csv([alpha, avg_quality], csvfile = path + 'alpha.csv')

alpha = 2, average quality = 0.4256934841898887
alpha = 4, average quality = 0.4329711483872398
alpha = 8, average quality = 0.4814026612697085
alpha = 16, average quality = 0.48739654471518207
alpha = 32, average quality = 0.4808492511931978
alpha = 64, average quality = 0.4864363478493822
alpha = 128, average quality = 0.48656338194070664


### Experiment: beta

In [10]:
# beta=0.01 from test above
bot_model.save_csv([0.01, 0.49038907607976123], csvfile = path + 'beta.csv')
for beta in [0.0001, 0.0002, 0.0005, 0.001, 0.002, 0.005, 0.02, 0.05, 0.1, 0.2, 0.5]:
    follower_net = bot_model.init_net(False, human_network = path + 'follower_network.gml', beta=beta, gamma=0.001)
    avg_quality = bot_model.simulation(False, network=follower_net, verbose=True, mu=0.5, phi=1, alpha=15)
    print('beta = {}, average quality = {}'.format(beta, avg_quality))
    bot_model.save_csv([beta, avg_quality], csvfile = path + 'beta.csv')

time_steps =  0 , q =  200
time_steps =  1 , q =  0.49016340867722547
time_steps =  2 , q =  0.45829437186748717
time_steps =  3 , q =  0.4799020036710269
time_steps =  4 , q =  0.47670439345036053
time_steps =  5 , q =  0.46677149021511627
time_steps =  6 , q =  0.47824183854077396
time_steps =  7 , q =  0.4505853360937683
time_steps =  8 , q =  0.48244727451917013
time_steps =  9 , q =  0.44484935317534374
time_steps =  10 , q =  0.489071352951982
time_steps =  11 , q =  0.48038913408766487
time_steps =  12 , q =  0.48111187961706353
time_steps =  13 , q =  0.45674510987661693
time_steps =  14 , q =  0.4893808864443343
time_steps =  15 , q =  0.45306909926989913
time_steps =  16 , q =  0.4694298435823545
time_steps =  17 , q =  0.48579062542634593
time_steps =  18 , q =  0.4857083646219697
time_steps =  19 , q =  0.48868639152199034
time_steps =  20 , q =  0.46636995904023565
time_steps =  21 , q =  0.4986844744087157
time_steps =  22 , q =  0.48207011016941154
time_steps =  23 , q =

### Experiment: gamma

In [14]:
# gamma=0.001 from test above
bot_model.save_csv([0.001, 0.49038907607976123], csvfile = path + 'gamma.csv')
for gamma in [0.0001, 0.0002, 0.0005, 0.002, 0.005, 0.01, 0.02, 0.05, 0.1, 0.2, 0.5]:
    follower_net = bot_model.init_net(False, human_network = path + 'follower_network.gml', beta=0.01, gamma=gamma)
    avg_quality = bot_model.simulation(False, network=follower_net, verbose=False, mu=0.5, phi=1, alpha=15)
    print('gamma = {}, average quality = {}'.format(gamma, avg_quality))
    bot_model.save_csv([gamma, avg_quality], csvfile = path + 'gamma.csv')

gamma = 0.0001, average quality = 0.47590118989990104
gamma = 0.0002, average quality = 0.49110978933620475
gamma = 0.0005, average quality = 0.47974305280211854
gamma = 0.002, average quality = 0.47217610366333035
gamma = 0.005, average quality = 0.4803758950970987
gamma = 0.01, average quality = 0.44506330853745196
gamma = 0.02, average quality = 0.4291719118666293
gamma = 0.05, average quality = 0.3797217140462188
gamma = 0.1, average quality = 0.3530738102542187
gamma = 0.2, average quality = 0.30639434404347166
gamma = 0.5, average quality = 0.20927011102314436


### Experiment: phi

In [ ]:
# phi=1 from test above
#bot_model.save_csv([1, 0.49038907607976123], csvfile = path + 'phi.csv')
#follower_net = bot_model.init_net(False, human_network = path + 'follower_network.gml', beta=0.01, gamma=0.001)
for phi in [17,18,19]: #[2,4,8,16,32]:
    avg_quality = bot_model.simulation(False, network=follower_net, verbose=False, mu=0.5, phi=phi, alpha=15)
    print('phi = {}, average quality = {}'.format(phi, avg_quality))
    bot_model.save_csv([phi, avg_quality], csvfile = path + 'phi.csv')